In [1]:
import os
import re

import nltk

# download the stopwords and wordnet corpus
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords
stopword = stopwords.words('english')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


import gensim
from gensim.models import KeyedVectors
from gensim.utils import simple_preprocess   


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ravi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ravi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ravi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# change working directory
import os
print('Before chdir:', os.getcwd())

os.chdir('F:\\cosinesim\\') 
         
print('After chdir:', os.getcwd())

os.listdir()

Before chdir: C:\Users\Ravi
After chdir: F:\cosinesim


['A TUTORIAL ON DISTANCE METRIC LEARNING.pdf',
 'cosine similarity-Copy2.ipynb',
 'Cosine_Similarity_with_Glove.ipynb',
 'embeddings.xlsx',
 'fasttext_model300.mod',
 'fasttext_model300.mod.vectors.npy',
 'glovemodel.mod',
 'glovemodel.mod.vectors.npy',
 'Semantic-Similarity-of-Sentences-master.zip',
 'text_similarity_solution .ipynb',
 'w2vecmodel.mod',
 'w2vecmodel.mod.vectors.npy']

In [3]:
%%time
# Preprocessing 
def tokenize(text):
    
    # use library re to replace urls by token - urlplaceholder
#     text = re.sub(url_regex, 'urlplaceholder', text)
    
    # use library re to replace non ascii characters by a space
#     text = re.sub(non_ascii_regex, ' ', text)

    # use word_tokenize to tokenize the sentences
    tokens = word_tokenize(text)
    
    # instantiate an object of class WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()

    # use a list comprehension to lemmatize the tokens and remove the the stopwords
    clean_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stopword]

    # return the tokens
    return clean_tokens

Wall time: 0 ns


In [4]:
%%time
# data - sentences
query = "apple is fruit"
doc = "apple is phone"

Wall time: 0 ns


In [5]:
%%time
import re
query_tokens = tokenize(query)
print("query tokens", query_tokens)

doc_tokens = tokenize(doc)
print("doc tokens", doc_tokens)

query tokens ['apple', 'fruit']
doc tokens ['apple', 'phone']
Wall time: 2.84 s


In [6]:
%%time
# caluclate distance - glove
import gensim
glove_model = gensim.models.KeyedVectors.load("./glovemodel.mod")
glove_model.wmdistance(query_tokens, doc_tokens)

Wall time: 4.02 s


2.9931454658508296

In [7]:
%%time
# caluclate similarity - glove
import gensim
glove_model = gensim.models.KeyedVectors.load("./glovemodel.mod")
glove_model.n_similarity(query_tokens, doc_tokens)

Wall time: 2.83 s


0.7795816

In [8]:
%%time
# caluclate similarity - w2vec
import gensim
v2w_model = gensim.models.KeyedVectors.load("./w2vecmodel.mod")
v2w_model.n_similarity(query_tokens, doc_tokens)

Wall time: 1min 36s


0.7117652

In [9]:
%%time
# caluclate similarity - fasttext
import gensim
fasttext_model300 = gensim.models.KeyedVectors.load("./fasttext_model300.mod")
fasttext_model300.n_similarity(query_tokens, doc_tokens)

Wall time: 1min 54s


0.7750639

In [10]:
%%time
# from gensim.models import KeyedVectors
# from gensim.utils import simple_preprocess    

def tidy_sentence(sentence, vocabulary):
    return [word for word in simple_preprocess(sentence) if word in vocabulary]    

def compute_sentence_similarity(sentence_1, sentence_2, model_wv):
    vocabulary = set(model_wv.index2word)    
    tokens_1 = tidy_sentence(sentence_1, vocabulary)    
    tokens_2 = tidy_sentence(sentence_2, vocabulary)    
    return model_wv.n_similarity(tokens_1, tokens_2)

# wv = KeyedVectors.load('model.wv', mmap='r')
sim = compute_sentence_similarity('this is a sentence', 'this is also a sentence', v2w_model)
print(sim)

0.9596676
Wall time: 21.7 s
Compiler : 211 ms
Parser   : 620 ms


In [11]:
%%time
import gensim
from gensim.models import KeyedVectors
from gensim.utils import simple_preprocess   

def get_sentence_similarity(sentence_1, sentence_2, model_wv):
    tokens_1 = tokenize(sentence_1)    
    tokens_2 = tokenize(sentence_2)    
    return model_wv.n_similarity(tokens_1, tokens_2)

fasttext_model300 = gensim.models.KeyedVectors.load("./fasttext_model300.mod")
sim = get_sentence_similarity('apple is fruit', 'apple is phone', fasttext_model300)
print(sim)

0.7750639
Wall time: 1min 44s
Parser   : 258 ms


In [12]:
%%time
import gensim
from gensim.models import KeyedVectors
  
def get_sentence_similarity(sentence_1, sentence_2, model_wv):
    tokens_1 = tokenize(sentence_1)    
    tokens_2 = tokenize(sentence_2)    
    return model_wv.n_similarity(tokens_1, tokens_2)

glove_model = gensim.models.KeyedVectors.load("./glovemodel.mod")
sim = get_sentence_similarity('apple is fruit', 'apple is phone', glove_model)
print(sim)

0.7795816
Wall time: 51.9 s
Compiler : 538 ms
Parser   : 481 ms


In [15]:
%%time
import gensim
from gensim.models import KeyedVectors

def get_sentence_similarity(sentence_1, sentence_2, model_wv):
    tokens_1 = tokenize(sentence_1)    
    tokens_2 = tokenize(sentence_2)    
    return model_wv.n_similarity(tokens_1, tokens_2)


v2w_model = gensim.models.KeyedVectors.load("./w2vecmodel.mod")
sim = get_sentence_similarity('apple is fruit', 'apple is phone', glove_model)
print(sim)

0.7795816
Wall time: 12min 14s


In [14]:
# def load_weModel(model, query, doc):
#     if model == 'glove':
#         glove_model = gensim.models.KeyedVectors.load("./glovemodel.mod")
#         print("Loaded glove model")
#     elif model == 'w2vec':
#         v2w_model = gensim.models.KeyedVectors.load("./w2vecmodel.mod")
#         print("Loaded w2v model")
#     elif model == 'fasttext':
#         fasttext_model300 = gensim.models.KeyedVectors.load("./fasttext_model300.mod")
#         print("Loaded fasttext_model300")
#     else:
#         print("Model not found")
        
# load_weModel('asdf')